In [1]:
# import libraries
import numpy as np
import pandas as pd
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [11]:
# datasets
train_df = pd.read_csv('input/train.csv')
test_df = pd.read_csv('input/test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [25]:
# example of a non-disaster tweet
train_df[train_df["target"] == 0]["text"].values[0]

"What's up man?"

In [26]:
# example of a disaster tweet
train_df[train_df["target"] == 1]["text"].values[0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [29]:
count_vectorizer = feature_extraction.text.CountVectorizer()

# we can look at the first 5 tweets in the data and get counts for them
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [43]:
# there are 54 unique words (or tokens) in the first 5 tweets
print(example_train_vectors[0].todense().shape)
# the first tweet contains only some of those unique tokens
print(example_train_vectors[0].todense())
# remaining questions, what are the rest of the components of example_train_vectors?

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]
